Используется старая реализация ESN, с новой еще не работал.

В MotionDetector исправить получение признаков и работу метода detect.

In [1]:
import os
import sys
import cv2
import numpy as np
from pathlib import Path

sys.path.append("../src")

from detector import MotionDetector
from model.esn_old import ESN

In [2]:
current_path = Path(os.getcwd())
project_path = current_path.parent
new_path = project_path / 'resources'

PATH_DATA = new_path

PATH_VIDEO = f"{PATH_DATA}/motion_video.avi"
# PATH_VIDEO = f"{PATH_DATA}/no_movement.avi"
# PATH_VIDEO = f"{PATH_DATA}/video_overexposure.avi"

In [3]:
model = ESN(
    input_size=10,
    reservoir_size=100,
    output_size=10,
    rhow=0.95,
    sparsity=0.2,
    leaky_rate=0.1,
    input_scaling=0.45
)

In [4]:
motion_detector = MotionDetector(
    lower_bound=np.array([64, 84, 75]),
    upper_bound=np.array([92, 255, 255]),
    model=model,
    min_area=1000,
    roi_algo="contours",
    contours_coords=[[639, 436], [464, 379], [200, 380], [0, 447], [0, 479], [170, 400], [482, 395], [636, 465]],
    cnt_zones=10,
    size_data_history=120,
    size_error_history=120,
    size_anomaly_history=4,
    score_threshold=0.5,
    factor=1.2
)

In [5]:
cap = cv2.VideoCapture(PATH_VIDEO)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = os.path.abspath('output_video.mp4')
out = cv2.VideoWriter(output_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

cnt = 0
temp = []
zone_masks = []

while cap.isOpened():
    
    ret, frame = cap.read()
    if not ret:
        break
    
    cnt += 1
    
    res = motion_detector.detect(frame)
    
    if not zone_masks:
        for zone in motion_detector.zones:
            zone_masks.append(zone["mask"])
    
    for mask in zone_masks:
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(frame, contours, -1, (0, 255, 0), 3)
    
    if res:
        temp.append(cnt)
        cv2.putText(frame, 'Anomaly', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    out.write(frame)

cap.release()
out.release()